# Emergency Planning

**Author**: Sebastian Granda

**Course**: Integer Programming

**References**
* Laporte, G., & Louveaux, F. V. (1993). The integer L-shaped method for stochastic integer programs with complete recourse.
* Birge, J. R., & Louveaux, F. (2011). Introduction to Stochastic Programming.

In this notebook we implement the L-shaped and Integer L-shaped methods in the iterative and callback versions using the multi-cut approach. The code can be found in the following [repository](https://github.com/SebastianGrandaA/EmergencyPlanning/).

To start, we use the following small instance to go through the process:

In [1]:
sample_instance = "EP_sites_5_scenarios_10_lf_0.5_seed_0";
sample_path = "$(pwd())/outputs/solutions/$(sample_instance)";

## Implementation notes

This project has been implemented as a Julia package called `EmergencyPlanning`. The main entry-point is the `execute` function, which creates an `Input` instance, solves the problem, and returns the solution as a `Solution` object. As part of the nomenclature, **all the optimization methods have been implemented with the `solve` function name**, using the multiple dispatch technique: `solve(::Base)`, `solve(::LShaped, ::Iterative)`, `solve(::LShaped, ::Callback)`, `solve(::IntegerLShaped)`. **All the results and solutions are stores in the `outputs` folder.

To set up this package, it is required to have `Gurobi.jl` installed with a valid license, otherwise replace `using Gurobi` and `Gurobi.Optimizer` from `src/EmergencyPlanning.jl` for an open source alternative, such as `GLPK.jl` :

```julia
using GLPK
solver = optimizer_with_attributes(GPLK.Optimizer)
```

All the necessary dependencies will be managed by running the following cell:

In [2]:
using Pkg
Pkg.activate(@__DIR__)
using EmergencyPlanning: benchmark!, test, execute, plot_summary!
using DataFrames
using CSV: read

  Activating project at `~/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning`


## Base model

The `Base` model allow us to compare the solutions obtained from the L-shaped methods against the original formulation of the problem description. Find this formulation at `src/methods/Base.jl`.

In [3]:
base_solution = execute(model="Base", filename=sample_instance, limit=60, benchmark=true);

┌ Info: Base | Instance EP_sites_5_scenarios_10_lf_0.5_seed_0 | Sites 5 | Teams 3 | Scenarios 10
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/EmergencyPlanning.jl:64
┌ Info: Solution is valid
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/services/helpers.jl:17


Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60


┌ Info: Instance EP_sites_5_scenarios_10_lf_0.5_seed_0 | Sites 5 | Teams 3 | Scenarios 10
│   metrics = 1×2 DataFrame
 Row │ objective_value  execution_time 
     │ Float64          Float64        
─────┼─────────────────────────────────
   1 │            10.0         1.02227
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/services/helpers.jl:117
┌ Info: Solution recorded in benchmark file
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/services/helpers.jl:86


The optimal solution of base model reports an objective value of `10` rescues:

In [4]:
read("$(sample_path)_Base_metrics.csv", DataFrame)

Row,objective_value,execution_time
,Float64,Float64
1,10.0,1.02227


For this instance, only team `1` is allocated at site `1`:

In [5]:
read("$(sample_path)_Base_allocations.csv", DataFrame)

Row,team,site
,String7,String7
1,TEAM-1,SITE-1


The following cell shows the first 5 rescues, which are team-to-site assignments, with their respective number of rescues, for a given scenario:

In [6]:
first(read("$(sample_path)_Base_rescues.csv", DataFrame), 5)

Row,scenario,team,site,nb_rescues
,String15,String7,String7,Float64
1,SCENARIO-1,TEAM-1,SITE-1,6.0
2,SCENARIO-1,TEAM-1,SITE-1,4.0
3,SCENARIO-2,TEAM-1,SITE-1,3.0
4,SCENARIO-2,TEAM-1,SITE-1,7.0
5,SCENARIO-3,TEAM-1,SITE-1,4.0


## L-shaped method

The L-shaped method is a decomposition technique for solving two-stage stochastic linear programs similar to Benders decomposition.
In the first stage the team location variables $x_{ik}$ are determined, while in the second stage the assignment of teams to sites $y^{s}_{ij}$ and the number of rescues $u^{s}_{ij}$ are determined for each scenario $s \in S$ independently.

The master problem provides candidate solutions and the subproblem either improves the lower bound (optimality cuts) or makes the problem feasible (feasibility cuts).
The master problem approximates the subproblem solutions by introducing an additional variable $\theta_{s} \forall s \in S$ that represents second-stage cost.
These cuts are added dynamically to the master problem using the dual solutions of the subproblems.

Given that the team location variables are fixed $\bar{x}_{ik}$, the subproblem solved to obtain the **optimality** cut is the following:

$$\text{SP-opt}^{s} = \quad \min - \sum_{i \in I} \sum_{j \in N(i)} u^{s}_{ij}$$

$$\text{s.t.} \quad \sum_{j \in N(i)} y^{s}_{ij} \leq 1 \quad \forall i \in I \quad \text{(exclusive assignment)}$$
$$\sum_{j \in N(i)} u^{s}_{ij} \leq d^{s}_{i} \quad \forall i \in I \quad \text{(demand satisfaction)}$$
$$\sum_{j \in N(i)} u^{s}_{ij} \leq \sum_{k \in K} C_{k} \bar{x}_{ik} \quad \forall i \in I \quad \text{(rescue capacity)}$$
$$u^{s}_{ij} \leq M y^{s}_{ij} \quad \forall i \in I, j \in N(i) \quad \text{(exclusive rescues)}$$
$$u^{s}_{ij} \geq 0 \quad \forall i \in I, j \in N(i)$$
$$y^{s}_{ij} \geq 0 \quad \forall i \in I, j \in N(i)$$

The objective function aims to maximize the total number of rescues by minimizing the negative number of rescues.
The exclusive assignment constraint ensures that rescue teams can be assigned to at most one neighboring site.
The demand satisfaction constraint guarantees that the number of rescues in each site does not exceed the demand.
The rescue capacity constraint provides an upper bound on the number of rescues in each site based on the capacity of the allocated teams.
The exclusive rescues constraint ensures that a team can rescue in a given site only if it is assigned to a neighboring site, using a large constant $M = \sum_{k \in K} C_{k}$.
The last two constraints define the domain of the variables $u^{s}_{ij}$ and $y^{s}_{ij}$, respectively.
Note that the integrality constraints of $y^{s}_{ij}$ has been relaxed to obtain the dual solutions.

Let $\lambda^{s}_{i}$ be the dual variable associated with the rescue capacity constraint, and define $v(s)$ as the objective value of the subproblem for scenario $s$.
As an assumption, we only consider the duals associated with the rescue capacity constraint, because it is the only constraint with relationship with the first-stage variables.
Then, the optimality cut for scenario $s$ is formulated as follows:

$$- \pi_{s} v(s) + \sum_{i \in I} \lambda^{s}_{i} \sum_{k \in K} C_{k} x_{ik} \leq \theta_{s}$$

Where $\pi_{s}$ is the probability of scenario $s$.
Due that the subproblem objective function is the only term in the master objective function, we add the optimality cut only if it improves the lower bound.
This cut is multplied by $-1$ to obtain the correct sign.

On the other hand, to obtain the **feasibility cut** we solve the auxiliary problem to determine the capacity deficit and surplus for each site $i \in I$.
Let $\delta^{+}_{i}$ and $\delta^{-}_{i}$ be the artificial variables associated with the capacity deficit and surplus, respectively.
Then, the auxiliary problem solved to obtain the feasibility cut is the following:

$$\text{SP-feas}^{s} = \quad \min \sum_{i \in I} \delta^{+}_{i} + \delta^{-}_{i}$$

$$\text{s.t.} \quad \sum_{j \in N(i)} y^{s}_{ij} \leq 1 \quad \forall i \in I \quad \text{(exclusive assignment)}$$
$$\sum_{j \in N(i)} u^{s}_{ij} \leq d^{s}_{i} \quad \forall i \in I \quad \text{(demand satisfaction)}$$
$$\sum_{j \in N(i)} u^{s}_{ij} + \delta^{+}_{i} - \delta^{-}_{i} \leq \sum_{k \in K} C_{k} \bar{x}_{ik} \quad \forall i \in I \quad \text{(rescue capacity)}$$
$$u^{s}_{ij} \leq M y^{s}_{ij} \quad \forall i \in I, j \in N(i) \quad \text{(exclusive rescues)}$$
$$u^{s}_{ij} \geq 0 \quad \forall i \in I, j \in N(i)$$
$$y^{s}_{ij} \geq 0 \quad \forall i \in I, j \in N(i)$$
$$\delta^{+}_{i} \geq 0 \quad \forall i \in I$$
$$\delta^{-}_{i} \geq 0 \quad \forall i \in I$$

The rescue capacity constraint now includes the artificial variables $\delta^{+}_{i}$ and $\delta^{-}_{i}$ to measure the infeasibility of the first-stage solution.
This auxiliary problem seeks to minimize the total capacity deficit and surplus, thus we consider the first-stage solution as feasible if all the artificial variables are zero.
Let $\beta^{s}_{i}$ be the dual variable associated with the rescue capacity constraint, and again consider $v(s)$ to represent the objective value for the feasibility subproblem in scenario $s$.
Then, the feasibility cut for scenario $s$ is formulated as follows:

$$- v(s) + \sum_{i \in I} \beta^{s}_{i} \sum_{k \in K} C_{k} x_{ik} \leq 0$$

The master problem is given by:

$$\text{(MP)} \quad \min - \sum_{s \in S} \pi_{s} \theta_{s}$$

$$\text{s.t.} \quad \sum_{k \in K} x_{ik} <= 1 \quad \forall i \in I \quad \text{(exclusive allocation)}$$
$$\sum_{k \in K} \sum_{i \in I} b_{k} x_{ik} \leq B \quad \text{(budget)}$$
$$\text{feasibility cuts}$$
$$\text{optimality cuts}$$
$$\theta_{s} \geq 0 \quad \forall s \in S$$
$$x_{ik} \in \{0, 1\} \quad \forall i \in I, k \in K$$

The objective function seeks to minimize the total expected recourse cost, which in the deterministic form is sum of the number of rescues in each scenario weighted by its probability.
The exclusive allocation constraint ensures that each team is allocated to at most one site.
The budget constraint ensures that the total cost of the team allocation does not exceed the budget limit $B$.

The multi-cut L-shaped method has been implemented in both the iterative and the callback versions.
The logic is orchestrated by the `solve!` function in either `src/methods/LShapedIterative.jl` or `src/methods/LShapedCallback.jl` files for each version. The master problem and subproblem formulations are located at `src/methods/problems/MasterProblem.jl` and `src/methods/problems/SubProblem.jl`, respectively, under the constructor functions named after them. In the master problem, there is a custom `solve!` function to ensure consistency of the iterations. In the subproblems, there are separate implementations for the optimality and feasibility subproblems.

For the iterative version, the subproblems are solved in parallel for each scenario; whereas for the callback version, the cuts are handled by the solver. The stop criteria for the iterative L-shaped method is based on the converage, which is the difference between the objective value of the master problem and the expected recourse, the maximum number of iterations and the maximum number of iterations without improvement.

To invoke the `solve` function of the iterative L-shaped method located at `src/methods/LShaped.jl`, we call:

In [7]:
ls_solution = execute(model="LShaped", usage=:iterative, filename=sample_instance, limit=60, benchmark=true);

┌ Info: LShaped-Iterative | Instance EP_sites_5_scenarios_10_lf_0.5_seed_0 | Sites 5 | Teams 3 | Scenarios 10
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/EmergencyPlanning.jl:64
┌ Info: LShaped-Iterative | Iteration 1
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/LShapedIterative.jl:20


Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60


┌ Warning: Model not optimal
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/services/helpers.jl:8
┌ Info: MasterProblem solved and updated | Metrics objective_value: 9.999999999999999e30, execution_time: 0.000560313, expected_recourse: 0
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/problems/MasterProblem.jl:79


Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLi

┌ Info: LShaped-Iterative | Iteration 2
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/LShapedIterative.jl:20
┌ Info: MasterProblem solved and updated | Metrics objective_value: 10.0, execution_time: 0.000536434, expected_recourse: 0
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/problems/MasterProblem.jl:79


Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLi

┌ Info: MasterProblem solved and updated | Metrics objective_value: 10.0, execution_time: 0.000695457, expected_recourse: 0
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/problems/MasterProblem.jl:79
┌ Info: LShaped-Iterative | History
│   master.history = EmergencyPlanning.Metrics[EmergencyPlanning.Metrics(9.999999999999999e30, 0.000560313, 0), EmergencyPlanning.Metrics(10.0, 0.000536434, 0), EmergencyPlanning.Metrics(10.0, 0.000695457, 0)]
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/LShapedIterative.jl:30
┌ Info: Solution is valid
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/services/helpers.jl:17
┌ Info: Instance EP_sites_5_scenarios_10_lf_0.5_seed_0 | Sites 5 | Teams

Equivalent to the `Base` model, the optimal value using the L-shaped method is `10` rescues. In following sections we show that this solutions are not integer.

In [8]:
read("$(sample_path)_LShaped_metrics.csv", DataFrame)

Row,objective_value,execution_time
,Float64,Float64
1,10.0,13.7147


The allocations are the same as well:

In [9]:
read("$(sample_path)_LShaped_allocations.csv", DataFrame)

Row,team,site
,String7,String7
1,TEAM-1,SITE-1


And the rescues too:

In [10]:
first(read("$(sample_path)_LShaped_rescues.csv", DataFrame), 5)

Row,scenario,team,site,nb_rescues
,String15,String7,String7,Float64
1,SCENARIO-1,TEAM-1,SITE-1,1.0
2,SCENARIO-2,TEAM-1,SITE-1,1.0
3,SCENARIO-3,TEAM-1,SITE-1,1.0
4,SCENARIO-4,TEAM-1,SITE-1,1.0
5,SCENARIO-5,TEAM-1,SITE-1,1.0


To use the L-shaped in the callback version:

In [11]:
ls_solution_c = execute(model="LShaped", usage=:callback, filename=sample_instance, limit=60, benchmark=true);

┌ Info: LShaped-Callback | Instance EP_sites_5_scenarios_10_lf_0.5_seed_0 | Sites 5 | Teams 3 | Scenarios 10
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/EmergencyPlanning.jl:64
┌ Warning: BORRAR
│   optimality_cut = JuMP.ScalarConstraint{JuMP.AffExpr, MathOptInterface.LessThan{Float64}}(-is_allocated[1,1] - 5 is_allocated[1,2] - 10 is_allocated[1,3] - is_allocated[2,1] - 5 is_allocated[2,2] - 10 is_allocated[2,3] - is_allocated[3,1] - 5 is_allocated[3,2] - 10 is_allocated[3,3] - is_allocated[4,1] - 5 is_allocated[4,2] - 10 is_allocated[4,3] - is_allocated[5,1] - 5 is_allocated[5,2] - 10 is_allocated[5,3] + θ[1], MathOptInterface.LessThan{Float64}(-0.0))
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/LShapedCallback.jl:47


┌ Info: LShaped-Callback | Scenario 1 | 1 cuts added | Master metrics
│   master.metrics = EmergencyPlanning.Metrics(9.999999999999999e30, NaN, 0)
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/LShapedCallback.jl:56
┌ Warning: BORRAR
│   optimality_cut = JuMP.ScalarConstraint{JuMP.AffExpr, MathOptInterface.LessThan{Float64}}(-is_allocated[1,1] - 5 is_allocated[1,2] - 10 is_allocated[1,3] - is_allocated[2,1] - 5 is_allocated[2,2] - 10 is_allocated[2,3] - is_allocated[3,1] - 5 is_allocated[3,2] - 10 is_allocated[3,3] - is_allocated[4,1] - 5 is_allocated[4,2] - 10 is_allocated[4,3] - is_allocated[5,1] - 5 is_allocated[5,2] - 10 is_allocated[5,3] + θ[2], MathOptInterface.LessThan{Float64}(-0.0))
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/LShapedCallback.jl:47
┌ Info: LShape

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLi

┌ Info: LShaped-Callback | Scenario 2 | 1 cuts added | Master metrics
│   master.metrics = EmergencyPlanning.Metrics(9.999999999999999e30, NaN, 0)
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/LShapedCallback.jl:56
┌ Warning: BORRAR
│   optimality_cut = JuMP.ScalarConstraint{JuMP.AffExpr, MathOptInterface.LessThan{Float64}}(-is_allocated[1,1] - 5 is_allocated[1,2] - 10 is_allocated[1,3] - is_allocated[2,1] - 5 is_allocated[2,2] - 10 is_allocated[2,3] - is_allocated[3,1] - 5 is_allocated[3,2] - 10 is_allocated[3,3] - is_allocated[4,1] - 5 is_allocated[4,2] - 10 is_allocated[4,3] - is_allocated[5,1] - 5 is_allocated[5,2] - 10 is_allocated[5,3] + θ[3], MathOptInterface.LessThan{Float64}(-0.0))
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/LShapedCallback.jl:47
┌ Info: LShape

┌ Info: LShaped-Callback | Scenario 4 | 1 cuts added | Master metrics
│   master.metrics = EmergencyPlanning.Metrics(9.999999999999999e30, NaN, 0)
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/LShapedCallback.jl:56
┌ Warning: BORRAR
│   optimality_cut = JuMP.ScalarConstraint{JuMP.AffExpr, MathOptInterface.LessThan{Float64}}(-is_allocated[1,1] - 5 is_allocated[1,2] - 10 is_allocated[1,3] - is_allocated[2,1] - 5 is_allocated[2,2] - 10 is_allocated[2,3] - is_allocated[3,1] - 5 is_allocated[3,2] - 10 is_allocated[3,3] - is_allocated[4,1] - 5 is_allocated[4,2] - 10 is_allocated[4,3] - is_allocated[5,1] - 5 is_allocated[5,2] - 10 is_allocated[5,3] + θ[5], MathOptInterface.LessThan{Float64}(-0.0))
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/LShapedCallback.jl:47
┌ Info: LShape

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLi

┌ Info: LShaped-Callback | Scenario 5 | 1 cuts added | Master metrics
│   master.metrics = EmergencyPlanning.Metrics(9.999999999999999e30, NaN, 0)
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/LShapedCallback.jl:56
┌ Warning: BORRAR
│   optimality_cut = JuMP.ScalarConstraint{JuMP.AffExpr, MathOptInterface.LessThan{Float64}}(-is_allocated[1,1] - 5 is_allocated[1,2] - 10 is_allocated[1,3] - is_allocated[2,1] - 5 is_allocated[2,2] - 10 is_allocated[2,3] - is_allocated[3,1] - 5 is_allocated[3,2] - 10 is_allocated[3,3] - is_allocated[4,1] - 5 is_allocated[4,2] - 10 is_allocated[4,3] - is_allocated[5,1] - 5 is_allocated[5,2] - 10 is_allocated[5,3] + θ[6], MathOptInterface.LessThan{Float64}(-0.0))
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/LShapedCallback.jl:47
┌ Info: LShape

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLi

┌ Info: LShaped-Callback | Scenario 3 | 1 cuts added | Master metrics
│   master.metrics = EmergencyPlanning.Metrics(9.999999999999999e30, NaN, 0)
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/LShapedCallback.jl:56
┌ Warning: BORRAR
│   optimality_cut = JuMP.ScalarConstraint{JuMP.AffExpr, MathOptInterface.LessThan{Float64}}(-is_allocated[1,1] - 5 is_allocated[1,2] - 10 is_allocated[1,3] - is_allocated[2,1] - 5 is_allocated[2,2] - 10 is_allocated[2,3] - is_allocated[3,1] - 5 is_allocated[3,2] - 10 is_allocated[3,3] - is_allocated[4,1] - 5 is_allocated[4,2] - 10 is_allocated[4,3] - is_allocated[5,1] - 5 is_allocated[5,2] - 10 is_allocated[5,3] + θ[4], MathOptInterface.LessThan{Float64}(1.0))
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/LShapedCallback.jl:47
┌ Info: LShaped

## Integer L-shaped method

Previouly, the integrality constraint of $y^{s}_{ij}$ has been relaxed.
In this section we explain the implementation of the Integer L-shaped method, which considers the integrality constraint. This method is implemented in the `src/methods/IntegerLShaped.jl` file, under the `solve` function, in the iterative version.

We start by solving the linear relaxation of the master problem. If the solution is integer, then we have found the optimal solution. Otherwise, we select a branching variable and create two nodes, one for each subproblem with the upper and lower bound constraints. This new subproblems are solved, evaluate if they should be pruned or not, and repeat the process until all nodes are pruned.

Therefore, the stop criteria is based on the quantity of pendant nodes and on the number of iterations. To determine whether a node should be pruned or not, we use three criteria: infeasibility, bound and integrality. In other words, we prune a node if it is infeasible or unbounded, or if the all variables are integer or if there exists a better solution (lower bound). The search criteria is breadth-first search, which means that we select the node with the best objective value over the set of pendant nodes. Finally, we use a random criteria to select the non-integer variable to branch on.

In [12]:
ils_solution = execute(model="IntegerLShaped", filename=sample_instance, limit=60, benchmark=true);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60


┌ Info: IntegerLShaped | Instance EP_sites_5_scenarios_10_lf_0.5_seed_0 | Sites 5 | Teams 3 | Scenarios 10
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/EmergencyPlanning.jl:64
┌ Info: LShaped-Iterative | Iteration 1
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/LShapedIterative.jl:20
┌ Warning: Model not optimal
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/services/helpers.jl:8
┌ Info: MasterProblem solved and updated | Metrics objective_value: 9.999999999999999e30, execution_time: 0.00031554, expected_recourse: 0
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/problems/Mas

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLi

┌ Info: LShaped-Iterative | Iteration 2
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/LShapedIterative.jl:20
┌ Info: MasterProblem solved and updated | Metrics objective_value: 10.0, execution_time: 0.000614396, expected_recourse: 0
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/problems/MasterProblem.jl:79


Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 60
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLi

┌ Info: MasterProblem solved and updated | Metrics objective_value: 10.0, execution_time: 0.000923108, expected_recourse: 0
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/problems/MasterProblem.jl:79
┌ Info: LShaped-Iterative | History
│   master.history = EmergencyPlanning.Metrics[EmergencyPlanning.Metrics(9.999999999999999e30, 0.00031554, 0), EmergencyPlanning.Metrics(10.0, 0.000614396, 0), EmergencyPlanning.Metrics(10.0, 0.000923108, 0)]
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/LShapedIterative.jl:30
┌ Info: Solution is valid
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/services/helpers.jl:17
┌ Info: Instance EP_sites_5_scenarios_10_lf_0.5_seed_0 | Sites 5 | Teams 

┌ Info: Solution recorded in benchmark file
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/services/helpers.jl:86


Validate that the metrics and the solution is the same as before:

In [13]:
read("$(sample_path)_IntegerLShaped_metrics.csv", DataFrame)

Row,objective_value,execution_time
,Float64,Float64
1,10.0,9.94823


In [14]:
read("$(sample_path)_IntegerLShaped_allocations.csv", DataFrame)

Row,team,site
,String7,String7
1,TEAM-1,SITE-1


In [15]:
read("$(sample_path)_IntegerLShaped_allocations.csv", DataFrame)

Row,team,site
,String7,String7
1,TEAM-1,SITE-1


## Numerical experiments

A `benchmark` function is created to execute the models with several instances and to compare them. The results of the benchmark are stores as a `benchmark.csv` file in the `outputs` folder. More details can be found at `src/services/benchmark.jl`.

For this analysis, we compare the `Base`, `LShaped` and `IntegerLShaped` methods with all the available instances in the `inputs` folder:

In [16]:
SETTINGS = Dict{String, Dict}(
    "Base" => Dict(),
    "LShaped" => Dict(:usage => :iterative),
    "IntegerLShaped" => Dict(),
);
MODELS = collect(keys(SETTINGS));
SAMPLE_SIZE = 1; # -1 to run all

We can run the benchmark for a given sample size of instances or for all of them (-1).

In [17]:
benchmark!(MODELS, SAMPLE_SIZE, SETTINGS)

┌ Info: Base | Instance EP_sites_50_scenarios_50_lf_0.8_seed_0 | Sites 50 | Teams 3 | Scenarios 50
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/EmergencyPlanning.jl:64
┌ Info: Solution is valid
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/services/helpers.jl:17


Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600


┌ Info: Instance EP_sites_50_scenarios_50_lf_0.8_seed_0 | Sites 50 | Teams 3 | Scenarios 50
│   metrics = 1×2 DataFrame
 Row │ objective_value  execution_time 
     │ Float64          Float64        
─────┼─────────────────────────────────
   1 │           399.2         42.6262
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/services/helpers.jl:117
┌ Info: Solution recorded in benchmark file
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/services/helpers.jl:86


Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600


┌ Info: LShaped-Iterative | Instance EP_sites_50_scenarios_50_lf_0.8_seed_0 | Sites 50 | Teams 3 | Scenarios 50
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/EmergencyPlanning.jl:64
┌ Info: LShaped-Iterative | Iteration 1
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/LShapedIterative.jl:20
┌ Warning: Model not optimal
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/services/helpers.jl:8
┌ Info: MasterProblem solved and updated | Metrics objective_value: 5.0e31, execution_time: 0.000790168, expected_recourse: 0
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/problems/MasterProbl

...

┌ Info: MasterProblem solved and updated | Metrics objective_value: 400.0, execution_time: 0.026891718, expected_recourse: 0
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/problems/MasterProblem.jl:79
┌ Info: LShaped-Iterative | History
│   master.history = EmergencyPlanning.Metrics[EmergencyPlanning.Metrics(5.0e31, 0.000790168, 0), EmergencyPlanning.Metrics(400.0, 0.018996809, 0), EmergencyPlanning.Metrics(400.0, 0.026891718, 0)]
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/LShapedIterative.jl:30
┌ Info: Solution is valid
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/services/helpers.jl:17
┌ Info: Instance EP_sites_50_scenarios_50_lf_0.8_seed_0 | Sites 50 | Teams 3 | Scen

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set pa

┌ Info: LShaped-Iterative | Iteration 2
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/LShapedIterative.jl:20
┌ Info: MasterProblem solved and updated | Metrics objective_value: 400.0, execution_time: 0.021063069, expected_recourse: 0
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/problems/MasterProblem.jl:79


Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set pa

┌ Info: MasterProblem solved and updated | Metrics objective_value: 400.0, execution_time: 0.02557165, expected_recourse: 0
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/problems/MasterProblem.jl:79
┌ Info: LShaped-Iterative | History
│   master.history = EmergencyPlanning.Metrics[EmergencyPlanning.Metrics(5.0e31, 0.000872525, 0), EmergencyPlanning.Metrics(400.0, 0.021063069, 0), EmergencyPlanning.Metrics(400.0, 0.02557165, 0)]
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/LShapedIterative.jl:30
┌ Info: Solution is valid
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/services/helpers.jl:17
┌ Info: Instance EP_sites_50_scenarios_50_lf_0.8_seed_0 | Sites 50 | Teams 3 | Scenar

Now that the instances are solved using the specified methods, we can discuss the results:

In [21]:
plot_summary!();

┌ Info: Statistical Summary
│   stats = 142×10 DataFrame
 Row │ model_name         instance_name                      objective_value_min  objective_value_mean  objective_value_std  objective_value_max  execution_time_min  execution_time_mean  execution_time_std  execution_time_max 
     │ String31           String                             Float64              Float64               Float64              Float64              Float64             Float64              Float64             Float64            
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ Base               EP_sites_100_scenarios_100_lf_0.…               200.0                200.0            0.0                       200.0          17.8464              19.2863              2.03627            20.7262
   2 │ LShaped-iterative  EP_sites_100_s

It is interesting to notice that the **LShaped models were able to solve the hardest instances to optimality**, such as `EP_sites_100_scenarios_100_lf_0.5_seed_0` and `EP_sites_100_scenarios_100_lf_0.5_seed_10`, whereas **the `Base` model did not find a feasible solution** in the time limit of 30 minutes. For other instances, such as for `EP_sites_100_scenarios_100_lf_0.8_seed_0`, the iterative version of the L-shaped method found the optimal solution in 58 seconds, while the `Base` model in 1761 seconds.

These results show that the **L-shaped method outperforms the `Base` model in terms of solution time and quality**, specially for large instances. The Integer L-shaped method is also able to achieve similar results that the regular L-shaped, but it is slower due to the **branch-and-bound** process. Finally, the fact that the subproblems are solved in **parallel** also contributes to the speedup of the iterative version of the L-shaped method, compared to the callback version.

Finally, we can test that the methods provide the expected objective value by comparing a set of instances:

In [19]:
EXPECTED_RESULTS = Dict{String, Real}(
    "EP_sites_5_scenarios_10_lf_0.5_seed_0" => 10,
    "EP_sites_5_scenarios_100_lf_0.5_seed_10" => 10,
    "EP_sites_10_scenarios_50_lf_0.2_seed_10" => 10,
    "EP_sites_10_scenarios_10_lf_0.5_seed_0" => 20,
    "EP_sites_10_scenarios_100_lf_0.5_seed_10" => 20,
    "EP_sites_10_scenarios_1_lf_0.8_seed_10" => 60,
    "EP_sites_50_scenarios_50_lf_0.5_seed_0" => 220,
    "EP_sites_100_scenarios_100_lf_0.5_seed_0" => 500, # only L-shaped found solution
    "EP_sites_100_scenarios_10_lf_0.5_seed_0" => 500,
    "EP_sites_100_scenarios_100_lf_0.8_seed_0" => 800, # [iterative] 58seg | [base] 1761seg
)

errors = test(MODELS, EXPECTED_RESULTS, SETTINGS)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600


┌ Info: Base | Instance EP_sites_5_scenarios_10_lf_0.5_seed_0 | Sites 5 | Teams 3 | Scenarios 10
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/EmergencyPlanning.jl:64
┌ Info: Solution is valid
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/services/helpers.jl:17
┌ Info: Instance EP_sites_5_scenarios_10_lf_0.5_seed_0 | Sites 5 | Teams 3 | Scenarios 10
│   metrics = 1×2 DataFrame
 Row │ objective_value  execution_time 
     │ Float64          Float64        
─────┼─────────────────────────────────
   1 │            10.0       0.0068375
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/services/helpers.jl:117
┌ Info: Solution recorded in benchmark file
└ @ EmergencyPlanning /Users/sebastiangranda

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set pa

┌ Info: LShaped-Iterative | Iteration 2
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/LShapedIterative.jl:20
┌ Info: MasterProblem solved and updated | Metrics objective_value: 10.0, execution_time: 0.000621019, expected_recourse: 0
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/problems/MasterProblem.jl:79


Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set pa

...

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set pa

┌ Info: LShaped-Iterative | Iteration 2
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/LShapedIterative.jl:20
┌ Info: MasterProblem solved and updated | Metrics objective_value: 10.0, execution_time: 0.001049902, expected_recourse: 0
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/problems/MasterProblem.jl:79


Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set pa

┌ Info: MasterProblem solved and updated | Metrics objective_value: 10.0, execution_time: 0.001164593, expected_recourse: 0
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/problems/MasterProblem.jl:79
┌ Info: LShaped-Iterative | History
│   master.history = EmergencyPlanning.Metrics[EmergencyPlanning.Metrics(9.999999999999999e30, 0.000448598, 0), EmergencyPlanning.Metrics(10.0, 0.001049902, 0), EmergencyPlanning.Metrics(10.0, 0.001164593, 0)]
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/methods/LShapedIterative.jl:30
┌ Info: Solution is valid
└ @ EmergencyPlanning /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/1. Integer programming/Projet/EmergencyPlanning/src/services/helpers.jl:17
┌ Info: Instance EP_sites_5_scenarios_10_lf_0.5_seed_0 | Sites 5 | Teams

Tuple{String, String}[]

In [20]:
isempty(errors) || error("Errors found: $errors")

true

In conclusion, this notebook described the implementation, benchmarking and testing/validating of the L-shaped method for the emergency planning problem. The L-shaped method, either in the iterative, callback or integer version, outperforms the `Base` model in terms of solution time and quality, specially for large instances.